# **GİRİŞ**
Bu çalışmada Vera Adalet Enstitüsü'nden sağlanan ABD hapishane verileri, veri görselleştirme teknikleri kullanılarak analiz edilmektedir. Çalışma boyunca 3 farklı soruya cevap aranmış, her cevap için farklı görselleştirme tekniğinden yararlanılmıştır. Benzer şekilde, ilgilenilen soruya göre veri setinin çalışmaya dahil edilen kısımları da farklılaşmaktadır.

#**1) ABD'de 2000-2014 yılları arasında  hapis cezası alan ortalama kişi sayısı eyaletlere göre farklılık göstermekte midir?**

Sorunun analizinde aşağıdaki veri tiplerinden yararlanılmıştır:

year: Yıl bilgisini vermektedir. Sıralı verilerden oluşmaktadır.                                 
total_prison_adm: Toplam hapis cezası alan kişi sayısını göstermektedir. Nicel verilerden oluşmaktadır.                                                                  
state: Eyalet bilgisini içermektedir. Kategorik verilerden oluşmaktadır.



## 1.1) Kod

In [ ]:
!pip install geopandas
!pip install plotly
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import json
import requests
import geopandas as gpd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display
url="https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2019/2019-01-22/incarceration_trends.csv"

data = pd.read_csv(url)
data_1 = data[(data['year'] >= 2000) & (data['year'] <= 2014)]
selected_columns = ['year', 'state','total_prison_adm']
data_1 = data_1[selected_columns]

data_1 = data_1.dropna(subset=['total_prison_adm']).groupby(["state","year"])[['total_prison_adm']].sum()
data_1 = data_1.groupby(["state"])[["total_prison_adm"]].mean().round(1)
data_1 = pd.DataFrame(data_1).reset_index()

#EYALETLERİN KOORDİNAT VERİLERİ YÜKLENMİŞTİR.
state = pd.read_csv('https://raw.githubusercontent.com/ozgeakindogan/yeni/main/USA2.csv')
data_1 = pd.merge(data_1, state, on='state', how='left')

fig = go.Figure()
fig.add_trace(go.Choropleth(
    locations=data_1['state'],
    locationmode="USA-states",
    z=data_1['total_prison_adm'],
    zmax=50000,
    zmin=0,
    colorscale="Reds",
    colorbar_title="Not: Renklendirilmeyen eyaletler için 2000-2014 yılları arasında veri bulunmamaktadır."+"<br>"+"<br>"+ "HAPİS CEZASI ALAN ORTALAMA KİŞİ SAYISI(2000-2014)",
    text="EYALET KODU: " + data_1['state'] + "<br>" +" HAPİS CEZASI ALAN ORTALAMA KİŞİ SAYISI(2000-2014): " + data_1['total_prison_adm'].astype(str),
    hovertemplate="%{text}<extra></extra>",
    marker_line_color='white'
))

fig.update_layout(
    dragmode=False,
    title="ABD Eyaletlerinde Hapis Cezası Alan Ortalama Kişi Sayısı (2000-2014)"
)
fig.update_geos(scope="usa")
fig.show()


## 1.2) Hapishane Verilerinin ABD Haritası Üzerinde Gösterimi

In [402]:
fig.show()

## 1.3) Görselleştirmede Kullanılan Veri İşaretleri

*   Eyalet sınırları
*   İmlecin harita üzerindeki hareketi ile veriye ait etiket bilgisinin görüntülenmesi

## 1.4) Görselleştirmede Kullanılan Veri Kanalları

*   Renk: Verinin ait olduğu değer aralığını ifade etmektedir. Sayının büyüklüğü arttıkça onu temsil eden renk de koyulaşmaktadır.
*   Renk barı: Renklerin değişim aralığını göstermektedir.
*   Harita: Eyaletlerin konumlarını göstermektedir.


## 1.5) Bulgular

2000-2014 yılları arası hapis cezası alan ortalama kişi sayısının en yüksek olduğu eyaletler Teksas ve Kaliforniya'dır. Bu eyaletlerde ortalama 50 binden fazla kişinin hapis cezası aldığı görülmektedir. Ceza sayısında Teksas ve Kaliforniya'yı sırasıyla Florida, Illinois, New York ve Ohio izlemektedir. Hapis cezası alan ortalama kişi sayısının en düşük olduğu eyaletler ise haritanın kuzey batısında yoğunlaşmaktadır. Eyaletlerin hukuk sistemine dair yeterli bilgiye sahip olunmadığı için hapis cezası alan kişi sayılarının azlığı/çokluğu ile eyaletin güvenliği arasında kesin bir ilişki kurulamamıştır.




# **2) 1989-2014 Yılları arasında Ohio, New York ve Florida eyaletlerinde hapis yatan kişi sayısı yıllara göre nasıl bir değişim göstermektedir?**

Sorunun analizimde aşağıdaki veri tiplerinden yararlanılmıştır:

year: Yıl bilgisini vermektedir. Sıralı verilerden oluşmaktadır.      
state: Eyalet bilgisini içermektedir. Kategorik verilerden oluşmaktadır.   
total_prison_pop: Hapishanede bulunan kişi sayısını ifade etmektedir. Nicel verilerden oluşmaktadır.


## 2.1) Kod

In [ ]:
states = ['OH', 'FL','NY']
data_2 = data[(data['year'] >= 1989) & (data['year'] <= 2014) & (data['state'].isin(states))]
data_2 =data_2.groupby(['year', 'state'])[['total_prison_pop']].sum()
data_2 = pd.DataFrame(data_2).reset_index()

years = data_2['year'].unique()
states = data_2['state'].unique()
total_prison_pop = data_2['total_prison_pop']
pop_min = np.min(total_prison_pop)
pop_max = np.max(total_prison_pop)
aralık_sayısı = 20
#Nokta büyüklükleri ayarlanır.
point_size_min = 5 
point_size_max = 45  
#Her veri aralığı artan büyüklükte noktalar ile temsil edilir.
pop_interval = (pop_max - pop_min) / (aralık_sayısı)
pop_intervals = [pop_min + i * pop_interval for i in range(aralık_sayısı+1)]

fig = go.Figure()
for state in states:
    state_data = data_2[data_2['state'] == state]
    state_pop = state_data['total_prison_pop']
    size_index = np.digitize(state_pop, pop_intervals) - 1
    point_sizes = [point_size_min + (point_size_max- point_size_min) * i / aralık_sayısı for i in size_index]
    state_data['total_prison_pop'] = state_data['total_prison_pop'].round().astype(int)
    fig.add_trace(go.Scatter(
        x=state_data['year'],
        y=[state] * len(state_data),
        mode='markers',
        marker=dict(
            size=point_sizes,
            sizemode='diameter',
            sizeref = 1.0,
            color='red',
            opacity=0.7
        ),
    text=state_data['total_prison_pop'],
    hovertemplate='Hapis Yatan Kişi Sayısı: %{text}',
    showlegend=False
    ))

tick_values = list(range(min(years), max(years) + 1))
fig.update_layout(
    title='Ohio, New York ve Florida Eyaletlerinde Hapis Yatan Kişi Sayısının Yıllara Göre Değişimi (1989-2014) ',
    xaxis=dict(
        title='YIL',
        tickmode='array',
        tickvals=tick_values,
    ),
    yaxis=dict(title='EYALET')
)

fig.show()

## 2.2) Hapishane Verilerinin Zamana Bağlı Değişiminin Şekil Üzerinde Gösterimi

In [404]:
fig.show()

## 2.3) Görselleştirmede Kullanılan Veri İşaretleri# 

*   Daire
*   İmlecin harita üzerindeki hareketi ile veriye ait etiket bilgisinin görüntülenmesi


## 2.4) Görselleştirmede Kullanılan Veri Kanalları

*   Dairelerin alanı: Kişi sayısının büyüklüğünü ifade etmektedir.
*   X ekseni: Yılları göstermektedir.
*   Y ekseni: Eyaletleri göstermektedir.

## 2.5) Bulgular

2000 yılına kadar, New York eyaletinde hapis yatan kişi sayısı Florida eyaletinde hapis yatan kişi sayısından yüksekken 2000-2014 yılları arasında bu sıralama değişiklik göstermiştir. Her bir yılın en düşük verisi Ohio eyaletine aittir. Florida eyaletinde hapis yatan kişi sayısı genel olarak zamanla artış göstermiş, en yüksek değerini de 2010 yılında almıştır. New York'ta ise veriler genel olarak 1999'a kadar artış göstermekte, 1999 yılından sonra da -2006 yılı hariç- azalmaktadır. Ohio eyaletinde hapis yatan kişi sayısı 1997 yılına kadar sürekli bir artış göstermekte, 1997'den sonraki yıllarda ise azalıp/artarak düzensiz bir seyir izlemektedir.

#**3) 1994-2014 Yılları arasında ABD'de raporlanmış cinayet, tecavüz, hırsızlık ve kundaklama suçlarının sayısı yıllara ve eyaletlere göre nasıl farklılık göstermektedir?**

Sorunun analizinde aşağıdaki veri tiplerinden yararlanılmıştır:

year: Yıl bilgisini vermektedir. Sıralı verilerden oluşmaktadır.                                 
robbery_crime: Raporlanan hırsızlık suçu sayısını ifade etmektedir. Nicel verilerden oluşmaktadır.            
murder_crime: Raporlanan cinayet suçu sayısını ifade etmektedir. Nicel verilerden oluşmaktadır.            
arson_crime: Raporlanan kundaklama suçu sayısını ifade etmektedir. Nicel verilerden oluşmaktadır.  
rape_crime: Raporlanan tecavüz suçu sayısını ifade etmektedir. Nicel verilerden oluşmaktadır.                                                                
state: Eyalet bilgisini içermektedir. Kategorik verilerden oluşmaktadır.




## 3.1) Kod

In [ ]:
data_3 = data[(data['year'] >= 1994) & (data['year'] <= 2014)]
selected_columns = ['year', 'state','murder_crime','rape_crime','robbery_crime','arson_crime']
data_3 = data_3[selected_columns]

data_3 = data_3.groupby(["state","year"])[['murder_crime','arson_crime','rape_crime','robbery_crime']].sum()
data_3 = pd.DataFrame(data_3).reset_index()

suc_turleri = ['murder_crime','arson_crime','rape_crime','robbery_crime']
state_dropdown = widgets.Dropdown(options=data_3['state'].unique(), description='Eyalet:')
year_dropdown = widgets.Dropdown(options=data_3['year'].unique(), description='Yıl:')
output = widgets.Output()

def update_plot(*args):
    selected_state = state_dropdown.value
    selected_year = year_dropdown.value
    selected_data = data_3[(data_3['state'] == selected_state) & (data_3['year'] == selected_year)]
    values = selected_data[suc_turleri].values.flatten()
    y_pos = range(len(suc_turleri))
    x_pos = range(len(values))
    output.clear_output(wait=True)  
    with output:
        plt.figure(figsize=(8, 4))
        bars = plt.barh(y_pos, values, color='lightcoral', height=0.2)
        plt.xlabel('Raporlanmış Suç Sayısı')
        plt.ylabel('Suç Türü')
        plt.title(selected_state + ' Eyaleti ' + str(selected_year) + ' Yılı İçin Suç Verileri')
        plt.yticks(y_pos, ['Cinayet', 'Kundaklama', 'Tecavüz', 'Hırsızlık'])  
        plt.tight_layout()
        
        for bar in bars:
            xval = bar.get_width()
            yval = bar.get_y() + bar.get_height() / 2
            plt.text(xval, yval, str(xval), ha='left', va='center', fontsize=8)
        
        plt.show()

state_dropdown.observe(update_plot, 'value')
year_dropdown.observe(update_plot, 'value')

update_plot()  
display(widgets.VBox([state_dropdown, year_dropdown, output]))


## 3.2) ABD Eyaletlerinde Raporlanmış Suç Verilerinin Zamana ve Eyaletlere Göre Değişiminin Şekil Üzerinde Gösterimi

In [406]:
state_dropdown.observe(update_plot, 'value')
year_dropdown.observe(update_plot, 'value')
update_plot()  
display(widgets.VBox([state_dropdown, year_dropdown, output]))

## 3.3) Görselleştirmede Kullanılan Veri İşaretleri# 

*   Bar

## 3.4) Görselleştirmede Kullanılan Veri Kanalları

*   X ekseni: Raporlanmış suç sayısını göstermektedir.
*   Y ekseni: Suç türlerini göstermektedir.
*   Seçim menüsü: Eyalet ve yıl bilgilerinin seçimi yapılmaktadır.

## 3.5) Bulgular

Verilerin neredeyse tamamında, raporlanmış hırsızlık suçu sayısı diğer 3 suç türünde raporlanmış vakalardan yüksek çıkmıştır. Suç türleri arasında en düşük verilerin de genel olarak cinayet suçuna ait olduğu gözlenmiştir. Wyoming eyaletinde raporlanmış tecavüz suçu sayısının diğer 3 suç türünden fazla olması dikkat çekmektedir.